In [1]:
pip install paho-mqtt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import requests
import json,os,csv
import pandas as pd
import os
os.environ["OPENELECTRICITY_API_KEY"] = "oe_3ZRd6d2R91fVxAEBekefybDa"
import paho.mqtt.client as mqtt
import json

# Task 1: Data Retrieval

## Task 1.1 Energy





In [2]:
# Define the normal fetch function to use, and get the facilities for use
import requests

API_KEY = "oe_3ZRd6d2R91fVxAEBekefybDa"
headers = {"Authorization": f"Bearer {API_KEY}"}

BASE = "https://api.openelectricity.org.au/v4"
NETWORK = "NEM"

START, END = "2025-10-01T00:00:00", "2025-10-08T00:00:00"

# general fetch
def fetch(url, params):
    r = requests.get(url, headers=headers, params=params, timeout=60)
    print(url, r.status_code)
    if r.ok:
        return r.json()
    else:
        print(r.text)
        return {}

facilities = fetch(f"{BASE}/facilities/", {})

nem_facilities = [
    f.get("code")
    for f in facilities.get("data", [])
    if f.get("network_id") == NETWORK
]

print("Total NEM facilities:", len(nem_facilities))
print(nem_facilities[:10])


https://api.openelectricity.org.au/v4/facilities/ 200
Total NEM facilities: 515
['ADP', 'ALDGASF', 'AMCORGR', 'ANGASTON', 'APS', 'APPIN', 'ARWF', 'AVLSF', 'AWABAREF', 'DEIBDL']


In [3]:
import json; json.dump(facilities, open("facilities_raw.json", "w", encoding="utf-8"), ensure_ascii=False, indent=2)

In [4]:
# Power data gettting one by one
import time

results_by_facility = {}

for code in nem_facilities:
    power = fetch(
        f"{BASE}/data/facilities/{NETWORK}",
        {
            "metrics": "energy",
            "interval": "5m",
            "facility_code": code,
            "date_start": START,
            "date_end": END
        }
    )
    results_by_facility[code] = power
    time.sleep(0.2)

print("Done. Total facilities fetched:", len(results_by_facility))

for code, data in list(results_by_facility.items())[:1]:
    print("\nSample facility:", code)
    print("Top-level keys:", list(data.keys()))

https://api.openelectricity.org.au/v4/data/facilities/NEM 200
https://api.openelectricity.org.au/v4/data/facilities/NEM 200
https://api.openelectricity.org.au/v4/data/facilities/NEM 416
{"version":"4.3.0","response_status":"ERROR","error":"No data available for facility ['AMCORGR'] in the specified time range","success":false}
https://api.openelectricity.org.au/v4/data/facilities/NEM 200
https://api.openelectricity.org.au/v4/data/facilities/NEM 416
{"version":"4.3.0","response_status":"ERROR","error":"No data available for facility ['APS'] in the specified time range","success":false}
https://api.openelectricity.org.au/v4/data/facilities/NEM 416
{"version":"4.3.0","response_status":"ERROR","error":"No data available for facility ['APPIN'] in the specified time range","success":false}
https://api.openelectricity.org.au/v4/data/facilities/NEM 200
https://api.openelectricity.org.au/v4/data/facilities/NEM 200
https://api.openelectricity.org.au/v4/data/facilities/NEM 416
{"version":"4.3.0",

KeyboardInterrupt: 

In [6]:
# remove not suitable datas and get the extra data from facilities json and get together into one good data set for using
facility_meta_map = {
    f["code"]: {
        "name": f.get("name", ""),
        "region_id": f.get("network_region", ""),
        "longitude": f.get("location", {}).get("lng"),
        "latitude": f.get("location", {}).get("lat")
    }
    for f in facilities.get("data", [])
    if f.get("network_id") == NETWORK
}

print("facility meta done:", len(facility_meta_map))

merged_data = []

for code, res in results_by_facility.items():
    if not res or not res.get("data"):
        continue
    meta = facility_meta_map.get(code, {})
    for record in res["data"]:
        merged_data.append({
            "facility_code": code,
            "facility_name": meta.get("name", ""),
            "region_id": meta.get("region_id", ""),
            "longitude": meta.get("longitude"),
            "latitude": meta.get("latitude"),
            **record
        })

import json

with open("NEM_energy_merged.json", "w", encoding="utf-8") as f:
    json.dump(merged_data, f, ensure_ascii=False, indent=2)

print("Done compiling and gathering", len(merged_data), "records.")


facility meta done: 515
Done compiling and gathering 335 records.


In [7]:
# transfer into df for futrue preprocessing
with open("NEM_energy_merged.json", "r", encoding="utf-8") as f:
    data = json.load(f)

records = []
for fac in data:
    code = fac.get("facility_code")
    name = fac.get("facility_name")
    region_id = fac.get("region_id")
    longitude = fac.get("longitude")
    latitude = fac.get("latitude")
    for res in fac.get("results", []):
        unit_code = res.get("columns", {}).get("unit_code")
        for ts, val in res.get("data", []):
            records.append({
                "facility_code": code,
                "facility_name": name,
                "unit_code": unit_code,
                "longitude": longitude,
                "latitude": latitude,
                "region_id": region_id,
                "timestamp": ts,
                "energy_MWh": val
            })

energy_df = pd.DataFrame(records)

print("DF transfer Done", len(energy_df), " of records")
print(energy_df.head(10))

DF transfer Done 979365  of records
  facility_code          facility_name unit_code   longitude   latitude  \
0           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
1           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
2           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
3           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
4           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
5           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
6           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
7           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
8           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
9           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   

  region_id                  timestamp  energy_MWh  
0       SA1  2025-10-01T00:00:00+10:00     -0.0003  
1       SA1  2025-10-01T00:05:00


## Task 1.2 Emissions




In [8]:
# Emissions data getting one by one
results_by_emissions = {}

for code in nem_facilities:
    emissions = fetch(
        f"{BASE}/data/facilities/{NETWORK}",
        {
            "metrics": "emissions",
            "interval": "5m",
            "facility_code": code,
            "date_start": START,
            "date_end": END
        }
    )
    results_by_emissions[code] = emissions
    time.sleep(0.2)

print("Done. Total facilities fetched (emissions):", len(results_by_emissions))

for code, data in list(results_by_emissions.items())[:1]:
    print("\nSample facility:", code)
    print("Top-level keys:", list(data.keys()))


https://api.openelectricity.org.au/v4/data/facilities/NEM 200
https://api.openelectricity.org.au/v4/data/facilities/NEM 200
https://api.openelectricity.org.au/v4/data/facilities/NEM 416
{"version":"4.3.0","response_status":"ERROR","error":"No data available for facility ['AMCORGR'] in the specified time range","success":false}
https://api.openelectricity.org.au/v4/data/facilities/NEM 200
https://api.openelectricity.org.au/v4/data/facilities/NEM 416
{"version":"4.3.0","response_status":"ERROR","error":"No data available for facility ['APS'] in the specified time range","success":false}
https://api.openelectricity.org.au/v4/data/facilities/NEM 416
{"version":"4.3.0","response_status":"ERROR","error":"No data available for facility ['APPIN'] in the specified time range","success":false}
https://api.openelectricity.org.au/v4/data/facilities/NEM 200
https://api.openelectricity.org.au/v4/data/facilities/NEM 200
https://api.openelectricity.org.au/v4/data/facilities/NEM 416
{"version":"4.3.0",

In [9]:
# remove not suitable data and merge emissions with facilities meta info
facility_meta_map = {
    f["code"]: {
        "name": f.get("name", ""),
        "region_id": f.get("network_region", ""),
        "longitude": f.get("location", {}).get("lng"),
        "latitude": f.get("location", {}).get("lat")
    }
    for f in facilities.get("data", [])
    if f.get("network_id") == NETWORK
}

print("facility meta done:", len(facility_meta_map))

merged_emissions = []

for code, res in results_by_emissions.items():
    if not res or not res.get("data"):
        continue
    meta = facility_meta_map.get(code, {})
    for record in res["data"]:
        merged_emissions.append({
            "facility_code": code,
            "facility_name": meta.get("name", ""),
            "region_id": meta.get("region_id", ""),
            "longitude": meta.get("longitude"),
            "latitude": meta.get("latitude"),
            **record
        })

import json

with open("NEM_emissions_merged.json", "w", encoding="utf-8") as f:
    json.dump(merged_data, f, ensure_ascii=False, indent=2)

print("Done compiling and gathering", len(merged_data), "records.")


facility meta done: 515
Done compiling and gathering 335 records.


In [10]:
# transfer emissions JSON into DataFrame for future preprocessing
with open("NEM_emissions_merged.json", "r", encoding="utf-8") as f:
    data = json.load(f)

records = []
for fac in data:
    code = fac.get("facility_code")
    name = fac.get("facility_name")
    region_id = fac.get("region_id")
    longitude = fac.get("longitude")
    latitude = fac.get("latitude")
    for res in fac.get("results", []):
        unit_code = res.get("columns", {}).get("unit_code")
        for ts, val in res.get("data", []):
            records.append({
                "facility_code": code,
                "unit_code": unit_code,
                "facility_name": name,
                "longitude": longitude,
                "latitude": latitude,
                "region_id": region_id,
                "timestamp": ts,
                "emissions_tCO2e": val   #
            })

emissions_df = pd.DataFrame(records)

print("DF transfer Done", len(emissions_df), "of emission records")
print(emissions_df.head(10))


DF transfer Done 979365 of emission records
  facility_code unit_code          facility_name   longitude   latitude  \
0           ADP    ADPBA1  Adelaide Desalination  138.484061 -35.096948   
1           ADP    ADPBA1  Adelaide Desalination  138.484061 -35.096948   
2           ADP    ADPBA1  Adelaide Desalination  138.484061 -35.096948   
3           ADP    ADPBA1  Adelaide Desalination  138.484061 -35.096948   
4           ADP    ADPBA1  Adelaide Desalination  138.484061 -35.096948   
5           ADP    ADPBA1  Adelaide Desalination  138.484061 -35.096948   
6           ADP    ADPBA1  Adelaide Desalination  138.484061 -35.096948   
7           ADP    ADPBA1  Adelaide Desalination  138.484061 -35.096948   
8           ADP    ADPBA1  Adelaide Desalination  138.484061 -35.096948   
9           ADP    ADPBA1  Adelaide Desalination  138.484061 -35.096948   

  region_id                  timestamp  emissions_tCO2e  
0       SA1  2025-10-01T00:00:00+10:00          -0.0003  
1       SA1  2

## Task 1.3 Market


In [11]:
# Use the regions code in the asignment specifications, to get data from "network" level
results_by_market = {}
regions = ["QLD1", "NSW1", "VIC1", "SA1", "TAS1"]

for region in regions:
    market_data = {}

    # ---- Price ----
    price = fetch(
        f"{BASE}/market/network/{NETWORK}",
        {
            "metrics": "price",
            "interval": "5m",
            "date_start": START,
            "date_end": END,
            "network_region": region
        }
    )
    market_data["price"] = price

    # ---- Demand ----
    demand = fetch(
        f"{BASE}/market/network/{NETWORK}",
        {
            "metrics": "demand",
            "interval": "5m",
            "date_start": START,
            "date_end": END,
            "network_region": region
        }
    )
    market_data["demand"] = demand

    results_by_market[region] = market_data
    time.sleep(0.2)

print("Done. Total regions fetched (market data):", len(results_by_market))

for region, data in list(results_by_market.items())[:1]:
    print("\nSample region:", region)
    print("Top-level keys:", list(data.keys()))


https://api.openelectricity.org.au/v4/market/network/NEM 200
https://api.openelectricity.org.au/v4/market/network/NEM 200
https://api.openelectricity.org.au/v4/market/network/NEM 200
https://api.openelectricity.org.au/v4/market/network/NEM 200
https://api.openelectricity.org.au/v4/market/network/NEM 200
https://api.openelectricity.org.au/v4/market/network/NEM 200
https://api.openelectricity.org.au/v4/market/network/NEM 200
https://api.openelectricity.org.au/v4/market/network/NEM 200
https://api.openelectricity.org.au/v4/market/network/NEM 200
https://api.openelectricity.org.au/v4/market/network/NEM 200
Done. Total regions fetched (market data): 5

Sample region: QLD1
Top-level keys: ['price', 'demand']


In [12]:
with open("NEM_market_data.json", "w", encoding="utf-8") as f:
    json.dump(results_by_market, f, ensure_ascii=False, indent=2)

print("Done compiling and gathering", len(results_by_market), "records.")

Done compiling and gathering 5 records.


In [13]:

import json
import pandas as pd

with open("NEM_market_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

records = []
for region, metrics in data.items():
    for metric_name, metric_data in metrics.items():
        for block in metric_data.get("data", []):
            for res in block.get("results", []):
                for ts, val in res.get("data", []):
                    records.append({
                        "region_id": region,
                        "timestamp": ts,
                        "metric": metric_name,
                        "metric_value": val
                    })

market_df = pd.DataFrame(records)

print("DF transfer Done", len(market_df), "of market records")
print(market_df.head(10))



DF transfer Done 20160 of market records
  region_id                  timestamp metric  metric_value
0      QLD1  2025-10-01T00:00:00+10:00  price         54.82
1      QLD1  2025-10-01T00:05:00+10:00  price         67.30
2      QLD1  2025-10-01T00:10:00+10:00  price         54.96
3      QLD1  2025-10-01T00:15:00+10:00  price         73.95
4      QLD1  2025-10-01T00:20:00+10:00  price         63.60
5      QLD1  2025-10-01T00:25:00+10:00  price         54.96
6      QLD1  2025-10-01T00:30:00+10:00  price         54.96
7      QLD1  2025-10-01T00:35:00+10:00  price         54.96
8      QLD1  2025-10-01T00:40:00+10:00  price         54.96
9      QLD1  2025-10-01T00:45:00+10:00  price         56.11


# Task 2: Data Integration and Materialization/Caching

In [14]:
fixed_market_df = market_df
fixed_emissions_df = emissions_df
fixed_energy_df = energy_df



In [15]:
# checking availability of emissions, the data is very clean
df = fixed_emissions_df.copy()
print(len(df))

df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df = df.drop_duplicates()
df.columns = df.columns.str.lower()

print(df.isna().sum())
fixed_emissions_df = df.copy()
print(fixed_emissions_df.head(10))


979365
facility_code      0
unit_code          0
facility_name      0
longitude          0
latitude           0
region_id          0
timestamp          0
emissions_tco2e    0
dtype: int64
  facility_code unit_code          facility_name   longitude   latitude  \
0           ADP    ADPBA1  Adelaide Desalination  138.484061 -35.096948   
1           ADP    ADPBA1  Adelaide Desalination  138.484061 -35.096948   
2           ADP    ADPBA1  Adelaide Desalination  138.484061 -35.096948   
3           ADP    ADPBA1  Adelaide Desalination  138.484061 -35.096948   
4           ADP    ADPBA1  Adelaide Desalination  138.484061 -35.096948   
5           ADP    ADPBA1  Adelaide Desalination  138.484061 -35.096948   
6           ADP    ADPBA1  Adelaide Desalination  138.484061 -35.096948   
7           ADP    ADPBA1  Adelaide Desalination  138.484061 -35.096948   
8           ADP    ADPBA1  Adelaide Desalination  138.484061 -35.096948   
9           ADP    ADPBA1  Adelaide Desalination  138.484061 -

In [16]:
# same as emissions
df = fixed_energy_df.copy()
print(len(df))

df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df = df.drop_duplicates()
print(len(df))
df.columns = df.columns.str.lower()

print(df.isna().sum())
fixed_energy_df = df.copy()
print(fixed_energy_df.head(10))


979365
979365
facility_code    0
facility_name    0
unit_code        0
longitude        0
latitude         0
region_id        0
timestamp        0
energy_mwh       0
dtype: int64
  facility_code          facility_name unit_code   longitude   latitude  \
0           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
1           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
2           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
3           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
4           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
5           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
6           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
7           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
8           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
9           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948

In [17]:
# market
df = fixed_market_df.copy()
print(len(df))
print(df.head(10))

df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df = df.drop_duplicates()
print(len(df))
print(df.head(10))
df.columns = df.columns.str.lower()
print()
print(df.isna().sum())
print()
print(df["metric"].value_counts())
fixed_market_df = df.copy()

20160
  region_id                  timestamp metric  metric_value
0      QLD1  2025-10-01T00:00:00+10:00  price         54.82
1      QLD1  2025-10-01T00:05:00+10:00  price         67.30
2      QLD1  2025-10-01T00:10:00+10:00  price         54.96
3      QLD1  2025-10-01T00:15:00+10:00  price         73.95
4      QLD1  2025-10-01T00:20:00+10:00  price         63.60
5      QLD1  2025-10-01T00:25:00+10:00  price         54.96
6      QLD1  2025-10-01T00:30:00+10:00  price         54.96
7      QLD1  2025-10-01T00:35:00+10:00  price         54.96
8      QLD1  2025-10-01T00:40:00+10:00  price         54.96
9      QLD1  2025-10-01T00:45:00+10:00  price         56.11
20160
  region_id                 timestamp metric  metric_value
0      QLD1 2025-10-01 00:00:00+10:00  price         54.82
1      QLD1 2025-10-01 00:05:00+10:00  price         67.30
2      QLD1 2025-10-01 00:10:00+10:00  price         54.96
3      QLD1 2025-10-01 00:15:00+10:00  price         73.95
4      QLD1 2025-10-01 00:20:00+1

In [18]:
pivot_df = fixed_market_df.pivot_table(
    index=["region_id", "timestamp"],
    columns="metric",
    values="metric_value",
    aggfunc="first"
).reset_index()
# add unit
pivot_df = pivot_df.rename(columns={
    "price": "price($/MWh)",
    "demand": "demand(MW)"
})
pivot_df[["price($/MWh)", "demand(MW)"]] = pivot_df[["price($/MWh)", "demand(MW)"]].fillna(-1)



In [19]:
print(len(pivot_df))
print(pivot_df.head(10))
print(pivot_df.isna().sum())
fixed_market_df = pivot_df.copy()

10060
metric region_id                 timestamp  demand(MW)  price($/MWh)
0           NSW1 2025-10-01 00:00:00+10:00     7105.57         56.98
1           NSW1 2025-10-01 00:05:00+10:00     7170.68         80.01
2           NSW1 2025-10-01 00:10:00+10:00     7158.50         58.40
3           NSW1 2025-10-01 00:15:00+10:00     7177.80         80.01
4           NSW1 2025-10-01 00:20:00+10:00     7085.54         65.00
5           NSW1 2025-10-01 00:25:00+10:00     7092.79         63.83
6           NSW1 2025-10-01 00:30:00+10:00     7066.08         61.05
7           NSW1 2025-10-01 00:35:00+10:00     7055.68         57.13
8           NSW1 2025-10-01 00:40:00+10:00     7077.07         61.05
9           NSW1 2025-10-01 00:45:00+10:00     6990.77         57.01
metric
region_id       0
timestamp       0
demand(MW)      0
price($/MWh)    0
dtype: int64


In [20]:
# According to A2 specification, merged two df into one csv
fixed_energy_df.columns = fixed_energy_df.columns.str.lower()
fixed_emissions_df.columns = fixed_emissions_df.columns.str.lower()

common_cols = list(set(fixed_energy_df.columns) & set(fixed_emissions_df.columns))

print("publics columns：", common_cols)

mixed_df = pd.merge(
    fixed_energy_df,
    fixed_emissions_df,
    on=common_cols,
    how="outer"
)

print("Merged successfully:", len(mixed_df), "records")
print(mixed_df.head(10))


publics columns： ['facility_name', 'latitude', 'region_id', 'facility_code', 'unit_code', 'timestamp', 'longitude']
Merged successfully: 979365 records
  facility_code          facility_name unit_code   longitude   latitude  \
0           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
1           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
2           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
3           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
4           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
5           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
6           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
7           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
8           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   
9           ADP  Adelaide Desalination    ADPBA1  138.484061 -35.096948   

  region_id           

In [21]:
mixed_df.to_csv("mixed_df.csv", index=False, encoding="utf-8")
fixed_market_df.to_csv("fixed_market_df.csv", index=False, encoding="utf-8")

In [2]:
import pandas as pd
import numpy as np
from typing import List

# Safe readers & utilities
def read_csv_safe(path: str) -> pd.DataFrame: 
    #Read CSV with low_memory=False and return empty DataFrame if file missing.
    try:
        return pd.read_csv(path, low_memory=False)
    except Exception as e:
        print(f"[WARN] Failed to read: {path} -> {e}")
        return pd.DataFrame()

def ensure_col(df: pd.DataFrame, col: str, fill=np.nan) -> pd.DataFrame:
    #Ensure a column exists; if missing, create with default fill value.
    if col not in df.columns:
        df[col] = fill
    return df

def normalize_name(s: pd.Series) -> pd.Series:
    #Lowercase, remove punctuation and extra spaces for name matching.
    return (s.astype(str)
              .str.lower()
              .str.replace(r"[\-\_\.\,\(\)\[\]\{\}\&\+\'\"]", " ", regex=True)
              .str.replace(r"\s+", " ", regex=True)
              .str.strip())

def normalize_code(s: pd.Series) -> pd.Series:
    #Uppercase and remove spaces for facility code matching.
    return (s.astype(str)
              .str.upper()
              .str.replace(r"\s+", "", regex=True)
              .str.strip())

def to_datetime_utc(s: pd.Series) -> pd.Series:
    #Parse timestamps to UTC, coercing invalid values to NaT.
    return pd.to_datetime(s, utc=True, errors="coerce")


# Read all five tables （Three from A1）

market = read_csv_safe("fixed_market_df.csv")
mixed  = read_csv_safe("mixed_df.csv")
cer    = read_csv_safe("cer_all.csv")
nger   = read_csv_safe("nger_with_geocode.csv")
absdf  = read_csv_safe("abs_table2.csv")  

# Standardize MARKET
if not market.empty:
    market = market.rename(columns={
        "region_id": "region",
        "timestamp": "timestamp",
        "demand(MW)": "demand_mw",
        "price($/MWh)": "price_$/mwh"
    })
    for c in ["region","timestamp","demand_mw","price_$/mwh"]:
        market = ensure_col(market, c)

# Standardize MIXED (main)
if mixed.empty:
    raise RuntimeError("[FATAL] mixed_df is required but could not be read.")

mixed = mixed.rename(columns={
    "region_id": "region",
    "timestamp": "timestamp",
    "lat": "latitude",
    "lon": "longitude"
}).copy()

# Build normalized keys for name+code matching
mixed["facility_code_norm"] = normalize_code(mixed.get("facility_code").fillna(""))
mixed["facility_name_norm"] = normalize_name(mixed.get("facility_name").fillna(""))

# Ensure columns used later exist
for c in ["region","timestamp","facility_code","facility_name","unit_code",
          "latitude","longitude","energy_mwh","emissions_tco2e"]:
    mixed = ensure_col(mixed, c)

# Standardize CER
cer_h = cer.rename(columns={
    "accreditation_code": "facility_code",
    "power_station_name": "facility_name",
    "fuel_source": "fueltype",
    "lat": "latitude",
    "lon": "longitude"
}).copy()
for c in ["facility_code","facility_name","fueltype","latitude","longitude"]:
    cer_h = ensure_col(cer_h, c)
cer_h["facility_code_norm"] = normalize_code(cer_h.get("facility_code").fillna(""))
cer_h["facility_name_norm"] = normalize_name(cer_h.get("facility_name").fillna(""))

# Standardize NGER
nger_h = nger.rename(columns={
    "primary_fuel": "fueltype",
    "lat": "latitude",
    "lon": "longitude"
}).copy()
for c in ["facility_name","fueltype","latitude","longitude"]:
    nger_h = ensure_col(nger_h, c)
nger_h["facility_name_norm"] = normalize_name(nger_h.get("facility_name").fillna(""))

# Primary join: (name + code) with CER
enriched = mixed.merge(
    cer_h[["facility_code_norm","facility_name_norm","fueltype","latitude","longitude"]].drop_duplicates(),
    on=["facility_code_norm","facility_name_norm"],
    how="left",
    suffixes=("", "_cer")
)

# Ensure fueltype column exists after merge
enriched = ensure_col(enriched, "fueltype")

# Prefer to keep original coords if present; otherwise fill from CER
for c in ["latitude","longitude"]:
    fill_col = f"{c}_cer"
    if fill_col in enriched.columns:
        enriched[c] = enriched[c].where(enriched[c].notna(), enriched[fill_col])

# Secondary join: by name only with NGER
nger_sel = nger_h[["facility_name_norm","fueltype","latitude","longitude"]].drop_duplicates()
nger_sel = ensure_col(nger_sel, "fueltype")
nger_sel = ensure_col(nger_sel, "latitude")
nger_sel = ensure_col(nger_sel, "longitude")

enriched = enriched.merge(
    nger_sel,
    on="facility_name_norm",
    how="left",
    suffixes=("", "_nger")
)

# Fill fueltype from NGER if still missing; coords as fallback
if "fueltype_nger" in enriched.columns:
    enriched["fueltype"] = enriched["fueltype"].fillna(enriched["fueltype_nger"])
for c in ["latitude","longitude"]:
    c_nger = f"{c}_nger"
    if c_nger in enriched.columns:
        enriched[c] = enriched[c].where(enriched[c].notna(), enriched[c_nger])

# Stronger name normalization and stopwords 
import re
STOPWORDS = {
    "power","station","plant","wind","solar","farm","hydro","gas","coal","diesel",
    "pty","ltd","limited","energy","generation","gen","co","company","facility","unit",
    "pv","battery","storage","project","trust"
}

def normalize_name_str(x: str) -> str:
    """Aggressive normalization: keep alnum + space, lowercase, collapse spaces, drop common suffix words."""
    x = str(x).lower()
    x = re.sub(r"[^a-z0-9\s]", " ", x)
    x = re.sub(r"\s+", " ", x).strip()
    tokens = [t for t in x.split() if t and t not in STOPWORDS]
    return " ".join(tokens)

def normalize_name(s: pd.Series) -> pd.Series:
    return s.fillna("").map(normalize_name_str)

def name_key_top2(x: str) -> str:
    """Build a short key from the first two normalized tokens to mitigate subtle variants."""
    toks = normalize_name_str(x).split()
    return " ".join(toks[:2]) if toks else ""

# Build canonical keys 
mixed["facility_name_norm"] = normalize_name(mixed.get("facility_name"))
mixed["facility_code_norm"] = normalize_code(mixed.get("facility_code").fillna(""))
mixed["name_key2"] = mixed["facility_name"].map(name_key_top2)

# CER canonical
cer_h["facility_name_norm"] = normalize_name(cer_h.get("facility_name"))
cer_h["facility_code_norm"] = normalize_code(cer_h.get("facility_code").fillna(""))
cer_h["name_key2"] = cer_h["facility_name"].map(name_key_top2)

# NGER canonical
nger_h["facility_name_norm"] = normalize_name(nger_h.get("facility_name"))
nger_h["name_key2"] = nger_h["facility_name"].map(name_key_top2)

# Ensure required columns exist
for df in (cer_h, nger_h):
    for c in ["region","fueltype","latitude","longitude"]:
        df = ensure_col(df, c)

# Reference union for name-based lookups
ref = pd.concat([
    cer_h[["region","facility_name_norm","name_key2","fueltype","latitude","longitude"]],
    nger_h[["region","facility_name_norm","name_key2","fueltype","latitude","longitude"]]
], ignore_index=True)

# Drop rows missing both name and coords
ref = ref.dropna(subset=["fueltype"])
ref["facility_name_norm"] = ref["facility_name_norm"].fillna("")
ref["name_key2"] = ref["name_key2"].fillna("")

# Choose a canonical fueltype per (region, facility_name_norm) by first occurrence
ref_name_reg = (ref
    .drop_duplicates(subset=["region","facility_name_norm"])
    .set_index(["region","facility_name_norm"])["fueltype"]
)

# Also build a global name map (no region)
ref_name_global = (ref
    .drop_duplicates(subset=["facility_name_norm"])
    .set_index("facility_name_norm")["fueltype"]
)

# Also a compact key map (region + name_key2) to catch abbreviated names
ref_key2_reg = (ref
    .query("name_key2 != ''")
    .drop_duplicates(subset=["region","name_key2"])
    .set_index(["region","name_key2"])["fueltype"]
)

# Start from mixed as the base
out = mixed.copy()
out = ensure_col(out, "fueltype")
out = ensure_col(out, "latitude")
out = ensure_col(out, "longitude")

# CER by code ONLY (exact)
# Map facility_code_norm to fueltype (prefer CER)
cer_code_map = (cer_h
    .query("facility_code_norm != ''")
    .drop_duplicates(subset=["facility_code_norm"])
    .set_index("facility_code_norm")["fueltype"]
)
mask = out["fueltype"].isna() & out["facility_code_norm"].ne("")
out.loc[mask, "fueltype"] = out.loc[mask, "facility_code_norm"].map(cer_code_map)

# Region + normalized name (CER→NGER) 
mask = out["fueltype"].isna() & out["facility_name_norm"].ne("")
pair_index = pd.MultiIndex.from_arrays([out.loc[mask, "region"].astype(str),
                                        out.loc[mask, "facility_name_norm"]],
                                       names=["region","facility_name_norm"])
out.loc[mask, "fueltype"] = ref_name_reg.reindex(pair_index).values

# Global normalized name (CER→NGER) 
mask = out["fueltype"].isna() & out["facility_name_norm"].ne("")
out.loc[mask, "fueltype"] = out.loc[mask, "facility_name_norm"].map(ref_name_global)

# Region + name_key2 (compact)
mask = out["fueltype"].isna() & out["name_key2"].ne("")
pair_index2 = pd.MultiIndex.from_arrays([out.loc[mask, "region"].astype(str),
                                         out.loc[mask, "name_key2"]],
                                        names=["region","name_key2"])
out.loc[mask, "fueltype"] = ref_key2_reg.reindex(pair_index2).values

# Region + geo multi-resolution (CER→NGER)
def build_geo_lookup(df: pd.DataFrame, nd=3):
    """Dict[(region,lat_r,lon_r)] -> fueltype at given rounding."""
    d = df.dropna(subset=["region","latitude","longitude","fueltype"]).copy()
    if d.empty: return {}
    d["lat_r"] = np.round(d["latitude"].astype(float), nd)
    d["lon_r"] = np.round(d["longitude"].astype(float), nd)
    d = d.drop_duplicates(subset=["region","lat_r","lon_r"])
    return {(r, la, lo): ft for r, la, lo, ft in zip(d["region"].astype(str), d["lat_r"], d["lon_r"], d["fueltype"])}

def geo_fill(out_df: pd.DataFrame, ref_df: pd.DataFrame, nd_seq=(3, 2, 2), expand=(1.0, 2.5, 5.0)):
    """
    Multi-resolution geo backfill within region.
    nd_seq: rounding decimals (3=0.001°, 2=0.01°).
    expand: km radii (~1km, 2.5km, 5km approx. for loose bins).
    """
    # Precompute radians per km (approx): 1 degree lat ~ 111 km
    # We simulate "radius bins" by widening the rounding threshold progressively.
    df = out_df
    for nd in nd_seq:
        # exact rounded match
        lookup = build_geo_lookup(ref_df, nd=nd)
        mask = df["fueltype"].isna() & df["latitude"].notna() & df["longitude"].notna()
        if not mask.any() or not lookup:
            continue
        lat_r = np.round(df.loc[mask, "latitude"].astype(float), nd)
        lon_r = np.round(df.loc[mask, "longitude"].astype(float), nd)
        reg   = df.loc[mask, "region"].astype(str)
        keys  = list(zip(reg, lat_r, lon_r))
        vals  = [lookup.get(k) for k in keys]
        ser   = pd.Series(vals, index=df.index[mask])
        df.loc[mask, "fueltype"] = df.loc[mask, "fueltype"].fillna(ser)

    # Very loose bin using 0.01° grid without exact key (approx radius)
    # We implement a second pass where we snap to a coarser grid and try neighbors.
    # This captures small coordinate drift without full k-d tree cost.
    nd = 2  # 0.01°
    coarse = ref_df.dropna(subset=["region","latitude","longitude","fueltype"]).copy()
    if coarse.empty:
        return df
    coarse["lat_r"] = np.round(coarse["latitude"].astype(float), nd)
    coarse["lon_r"] = np.round(coarse["longitude"].astype(float), nd)
    # Build a small group index per region/grid
    grp = coarse.groupby(["region","lat_r","lon_r"])["fueltype"].agg("first")

    mask = df["fueltype"].isna() & df["latitude"].notna() & df["longitude"].notna()
    if not mask.any():
        return df
    df["_lat_r2"] = np.round(df["latitude"].astype(float), nd)
    df["_lon_r2"] = np.round(df["longitude"].astype(float), nd)

    # Try 9-cell neighborhood (self + 8 neighbors)
    for dlat in (-0.01, 0, 0.01):
        for dlon in (-0.01, 0, 0.01):
            if dlat == 0 and dlon == 0:
                continue
            submask = df["fueltype"].isna() & mask
            if not submask.any():
                break
            keys = list(zip(df.loc[submask, "region"].astype(str),
                            df.loc[submask, "_lat_r2"] + dlat,
                            df.loc[submask, "_lon_r2"] + dlon))
            vals = [grp.get(k) if k in grp.index else None for k in keys]
            ser  = pd.Series(vals, index=df.index[submask])
            df.loc[submask, "fueltype"] = df.loc[submask, "fueltype"].fillna(ser)

    df.drop(columns=["_lat_r2","_lon_r2"], errors="ignore", inplace=True)
    return df

# CER first, then NGER
out = geo_fill(out, cer_h, nd_seq=(3,2))
out = geo_fill(out, nger_h, nd_seq=(3,2))

# Region-scoped Jaccard token similarity fallback 
def jaccard(a: str, b: str) -> float:
    sa, sb = set(str(a).split()), set(str(b).split())
    if not sa or not sb: return 0.0
    inter = len(sa & sb); union = len(sa | sb)
    return inter / union if union else 0.0

def backfill_by_name_similarity(merged_df: pd.DataFrame, ref_df: pd.DataFrame, threshold=0.82):
    # Build region buckets in ref
    r = ref_df.copy()
    r = r.dropna(subset=["fueltype"])
    r["facility_name_norm"] = r["facility_name_norm"].fillna("")
    buckets = {rg: g[["facility_name_norm","fueltype"]].drop_duplicates()
               for rg, g in r.groupby(r["region"].astype(str), dropna=False)}
    miss_idx = merged_df.index[merged_df["fueltype"].isna()]
    # Iterate rows where fueltype is missing; restrict to same region bucket
    for idx in miss_idx:
        rg = str(merged_df.at[idx, "region"])
        nm = merged_df.at[idx, "facility_name_norm"]
        pool = buckets.get(rg)
        if pool is None or pool.empty: 
            continue
        # Compute similarity; pick best ≥ threshold
        sims = pool["facility_name_norm"].map(lambda x: jaccard(nm, x))
        best_i = sims.idxmax()
        if pd.notna(best_i) and sims.loc[best_i] >= threshold:
            ft = pool.loc[best_i, "fueltype"]
            if pd.notna(ft):
                merged_df.at[idx, "fueltype"] = ft
    return merged_df

# Use CER+NGER union as reference for similarity
ref_for_sim = ref[["region","facility_name_norm","fueltype"]].dropna()
out = backfill_by_name_similarity(out, ref_for_sim, threshold=0.82)

# Merge MARKET (region + timestamp)
out["timestamp"] = to_datetime_utc(out["timestamp"])
if not market.empty:
    market["timestamp"] = to_datetime_utc(market["timestamp"])
    merged = out.merge(
        market[["region","timestamp","demand_mw","price_$/mwh"]],
        on=["region","timestamp"],
        how="left"
    )
else:
    merged = out.copy()
    merged = ensure_col(merged, "demand_mw")
    merged = ensure_col(merged, "price_$/mwh")

#  ABS light merge
if not absdf.empty and "region" in merged.columns:
    cand = [c for c in absdf.columns if any(k in c.lower() for k in ["region","state","code"])]
    abs_clean = absdf[cand].dropna(how="all")
    if "region" in abs_clean.columns:
        merged = merged.merge(abs_clean.drop_duplicates(), on="region", how="left")

# Select and save 
final_cols = [
    "region","timestamp","demand_mw","price_$/mwh",
    "facility_code","facility_name","unit_code",
    "longitude","latitude","energy_mwh","emissions_tco2e","fueltype"
]
for c in final_cols:
    merged = ensure_col(merged, c)

final_out = merged[final_cols].copy()
final_out.to_csv("final_merged.csv", index=False)

# Coverage report
total = len(final_out)
ft_covered = total - final_out["fueltype"].isna().sum()
coverage = ft_covered / total * 100.0
print("Done. Saved:", "final_merged.csv")
print("Fueltype coverage: %.2f%%   (%d / %d)" % (coverage, ft_covered, total))
print("Missing counts per column:\n", final_out.isna().sum())

# Top missing names for quick aliasing
print("\nTop 20 missing facility names:")
print(final_out.loc[final_out["fueltype"].isna(), "facility_name"].value_counts().head(20))

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_39180\2956646383.py:246: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  out.loc[mask, "fueltype"] = out.loc[mask, "facility_code_norm"].map(cer_code_map)


Done. Saved: final_merged.csv
Fueltype coverage: 58.10%   (569000 / 979365)
Missing counts per column:
 region                  0
timestamp               0
demand_mw            1932
price_$/mwh          1932
facility_code           0
facility_name           0
unit_code               0
longitude               0
latitude                0
energy_mwh              0
emissions_tco2e         0
fueltype           410365
dtype: int64

Top 20 missing facility names:
facility_name
Tarong                    12096
Valley Power Peaking      12096
Jeeralang A                8064
Yallourn W                 8064
Bell Bay Three             8064
Loy Yang A                 8064
Morganwhyalla Pipeline     8064
Jeeralang B                6048
Happy Valley               6048
Lake Bonney                6048
Golden Plains East         6048
Smithfield                 6048
Tailem Bend 2              6048
Adelaide Desalination      6048
Western Downs              6048
Tumut 3                    4032
Ulinda Park  

# Task 3: Data Publishing via MQTT

In [11]:
# PRE
# I seached about the USYD BROKER but find nothing. 172.17.34.107 is not accessable. So i pick the public address shown on the lecture
BROKER = "test.mosquitto.org"
PORT   = 1883
TOPIC  = "COMP5339/Comp5339A2_TUT19_Group08/energy_emission/qwan7612"

df = pd.read_csv("final_merged.csv")


In [ ]:
import json
import time
import paho.mqtt.client as mqtt

client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2, transport="websockets")
client.ws_set_options(path="/mqtt")
client.tls_set()        # if using wss (secure)
client.connect("test.mosquitto.org", 8081, 60)
client.loop_start()

for i, row in df.iterrows():
     
    # f i > 300:         # change!!!!!!!!!! limit 300 for test. If you wanto change, delete the whole if statement!!!!
        #break

    message = {
        "facility_code": row.get("facility_code"),
        "facility_name": row.get("facility_name"),
        "unit_code": row.get("unit_code"),
        "fueltype": row.get("fueltype"),
        "region_id": row.get("region"),
        "lat": row.get("latitude"),
        "lon": row.get("longitude"),
        "timestamp": str(row.get("timestamp")),
        "energy_MWh": row.get("energy_mwh"),
        "emissions_tCO2e": row.get("emissions_tco2e"),
        "spot_price": row.get("price_$/mwh"),
        "demand_mw": row.get("demand_mw")
    }

    result = client.publish(TOPIC, json.dumps(message), qos=1, retain=False)
    if result.rc == mqtt.MQTT_ERR_SUCCESS:
        print(f"[{i}] Published → {message['timestamp']}")
    else:
        print(f"[{i}] Publish failed (code: {result.rc})")

    time.sleep(0.2)

client.loop_stop()
client.disconnect()
print("Records published successfully.")


[0] Published → 2025-09-30 14:00:00+00:00
[1] Published → 2025-09-30 14:05:00+00:00
[2] Published → 2025-09-30 14:10:00+00:00
[3] Published → 2025-09-30 14:15:00+00:00
[4] Published → 2025-09-30 14:20:00+00:00
[5] Published → 2025-09-30 14:25:00+00:00
[6] Published → 2025-09-30 14:30:00+00:00
[7] Published → 2025-09-30 14:35:00+00:00
[8] Published → 2025-09-30 14:40:00+00:00
[9] Published → 2025-09-30 14:45:00+00:00
[10] Published → 2025-09-30 14:50:00+00:00
[11] Published → 2025-09-30 14:55:00+00:00
[12] Published → 2025-09-30 15:00:00+00:00
[13] Published → 2025-09-30 15:05:00+00:00
[14] Published → 2025-09-30 15:10:00+00:00
[15] Published → 2025-09-30 15:15:00+00:00
[16] Published → 2025-09-30 15:20:00+00:00
[17] Published → 2025-09-30 15:25:00+00:00
[18] Published → 2025-09-30 15:30:00+00:00
[19] Published → 2025-09-30 15:35:00+00:00
[20] Published → 2025-09-30 15:40:00+00:00
[21] Published → 2025-09-30 15:45:00+00:00
[22] Published → 2025-09-30 15:50:00+00:00
[23] Published → 2025

# Task 5： Continuous Execution

In [10]:
# Publishes data for each facility_code sequentially.
# Each facility (power station) waits 60 seconds before the next one starts.
# Each message has 0.1s delay (same as Task 3).
# Only updated power/emission values are published to MQTT (for Exceeds grade).

import pandas as pd
import json, time
import paho.mqtt.client as mqtt

BROKER = "test.mosquitto.org"
PORT   = 8081
TOPIC  = "COMP5339/Comp5339A2_TUT19_Group08/energy_emission/qwan7612"
KEEPALIVE = 60
DATA_PATH = "final_merged.csv"

# Keep previous power/emission values to detect updates
previous_values = {}

def publish_facility_data(df_facility, facility_code):
    # Publish only updated records for one facility
    client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2, transport="websockets")
    client.ws_set_options(path="/mqtt")
    client.tls_set()
    client.connect(BROKER, PORT, KEEPALIVE)
    client.loop_start()

    print(f"\n[Task5] >>> Publishing data for facility: {facility_code} ({len(df_facility)} records)")

    for i, row in df_facility.iterrows():
        key = (row.get("facility_code"), str(row.get("timestamp")))
        new_power = row.get("energy_mwh")
        new_emiss = row.get("emissions_tco2e")

        # Only send if new or updated
        if key in previous_values:
            old_power, old_emiss = previous_values[key]
            if new_power == old_power and new_emiss == old_emiss:
                continue  # skip unchanged record

        # Update record history
        previous_values[key] = (new_power, new_emiss)

        # Build message
        message = {
            "facility_code": row.get("facility_code"),
            "facility_name": row.get("facility_name"),
            "unit_code":     row.get("unit_code"),
            "fueltype":      row.get("fueltype"),
            "region_id":     row.get("region"),
            "lat":           row.get("latitude"),
            "lon":           row.get("longitude"),
            "timestamp":     str(row.get("timestamp")),
            "energy_MWh":    new_power,
            "emissions_tCO2e": new_emiss,
            "spot_price":    row.get("price_$/mwh"),
            "demand_mw":     row.get("demand_mw")
        }

        # Publish to MQTT
        result = client.publish(TOPIC, json.dumps(message), qos=1, retain=False)
        if result.rc == mqtt.MQTT_ERR_SUCCESS:
            print(f"[{i}] Published updated → {message['timestamp']}")
        else:
            print(f"[{i}] Publish failed (code: {result.rc})")

        time.sleep(0.1)  # 0.1s delay (Task 3 requirement)

    client.loop_stop()
    client.disconnect()
    print(f"[Task5] Facility {facility_code} completed successfully.")

# Continuous runner
try:
    df = pd.read_csv(DATA_PATH, low_memory=False)
    facilities = df["facility_code"].dropna().unique().tolist()

    round_idx = 0
    while True:
        for facility_code in facilities:
            round_idx += 1
            print(f"\n==============================")
            print(f"[Task5] Round {round_idx} — Facility: {facility_code}")
            print("==============================")

            # subset for this facility
            df_facility = df[df["facility_code"] == facility_code]

            publish_facility_data(df_facility, facility_code)

            print(f"[Task5] Finished {facility_code}. Waiting 60 seconds before next facility...\n")
            time.sleep(60)  # 60-second delay between facilities

except KeyboardInterrupt:
    print("\n[Task5] Stopped by user. Bye.")


[Task5] Round 1 — Facility: ADP

[Task5] >>> Publishing data for facility: ADP (6048 records)
[0] Published updated → 2025-09-30 14:00:00+00:00
[1] Published updated → 2025-09-30 14:05:00+00:00
[2] Published updated → 2025-09-30 14:10:00+00:00
[3] Published updated → 2025-09-30 14:15:00+00:00
[4] Published updated → 2025-09-30 14:20:00+00:00
[5] Published updated → 2025-09-30 14:25:00+00:00
[6] Published updated → 2025-09-30 14:30:00+00:00
[7] Published updated → 2025-09-30 14:35:00+00:00
[8] Published updated → 2025-09-30 14:40:00+00:00
[9] Published updated → 2025-09-30 14:45:00+00:00
[10] Published updated → 2025-09-30 14:50:00+00:00
[11] Published updated → 2025-09-30 14:55:00+00:00
[12] Published updated → 2025-09-30 15:00:00+00:00
[13] Published updated → 2025-09-30 15:05:00+00:00
[14] Published updated → 2025-09-30 15:10:00+00:00
[15] Published updated → 2025-09-30 15:15:00+00:00
[16] Published updated → 2025-09-30 15:20:00+00:00
[17] Published updated → 2025-09-30 15:25:00+00: